### [RLAIF vs. RLHF: Scaling Reinforcement Learning from Human Feedback with AI Feedback](https://arxiv.org/abs/2309.00267)

Nesse notebook temos a inteção de fazer uma reprodução simples do método explorado no artigo

In [ ]:
import sys
sys.path.append(r"C:\dev")
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

# carregar modelo pequeno (por isso, o resultado pode ser ruim, aqui seria interessante 
# usar um modelo maior)
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# exemplos fictícios
examples = [
    {
        "instruction": "Explique em poucas palavras o que é aprendizado por reforço.",
        "response_0": "É um método onde o agente aprende por tentativa e erro, recebendo recompensas ou punições.",
        "response_1": "É um tipo de aprendizado que sempre usa redes neurais convolucionais para imagens."
    },
    {
        "instruction": "Resuma a fábula da Cigarra e da Formiga.",
        "response_0": "A formiga trabalhou no verão, enquanto a cigarra cantava. No inverno, a formiga tinha comida e a cigarra passou fome.",
        "response_1": "A história fala sobre uma cigarra que gostava de futebol e uma formiga que viajava muito."
    }
]

# Prompt do artigo
def build_prompt(instruction, resp_a, resp_b):
    return (
        "A good summary is coherent, accurate, covers the main ideas, and maintains overall quality.\n"
        "You are an expert rater. Given a text and two possible summaries, output 1 or 2 to indicate which is better.\n\n"
        f"Text - {instruction}\n"
        f"Summary 1 - {resp_a}\n"
        f"Summary 2 - {resp_b}\n"
        f"Preferred Summary="
    )

# função de juiz usando logprobs
def llm_judge(instruction, resp_a, resp_b):
    # constroi o prompt e tokeniza
    prompt = build_prompt(instruction, resp_a, resp_b)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Aqui, ao invés de gerar texto, queremos pegar a probabilidade dos tokens "1" e "2"
    # A gente não usar o generate, só o forward e pegamos os logits
    with torch.no_grad():
        outputs = model(**inputs)   
        logits = outputs.logits[:, -1, :]  # último passo (próximo token)

    token_id_1 = tokenizer.encode("1", add_special_tokens=False)[0]
    token_id_2 = tokenizer.encode("2", add_special_tokens=False)[0]

    # Pontuação de probabilidades dos tokens "1" e "2"
    logit_1 = logits[0, token_id_1].item()
    logit_2 = logits[0, token_id_2].item()

    # probabilidades normalizadas
    probs = torch.softmax(torch.tensor([logit_1, logit_2]), dim=0).tolist()
    p1, p2 = probs

    # Comparando as duas probabilidades, temos então a chose e a rejected
    label = 1 if p1 > p2 else 2
    return {"label": label, "p1": p1, "p2": p2, "prompt": prompt}

# pipeline
records = []
for ex in examples:
    result = llm_judge(ex["instruction"], ex["response_0"], ex["response_1"])
    label = result["label"]

    chosen = ex["response_0"] if label == 1 else ex["response_1"]
    rejected = ex["response_1"] if label == 1 else ex["response_0"]

    records.append({
        "instruction": ex["instruction"],
        "chosen": chosen,
        "rejected": rejected,
        "p1": result["p1"],
        "p2": result["p2"],
        "prompt_used": result["prompt"]
    })

df = pd.DataFrame(records)
df


c:\dev\Track_RLAIF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\dev\Track_RLAIF\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moura\.cache\huggingface\hub\models--microsoft--phi-1_5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this artic

,instruction,chosen,rejected,p1,p2,prompt_used
0,Explique em poucas palavras o que é aprendizad...,É um método onde o agente aprende por tentativ...,É um tipo de aprendizado que sempre usa redes ...,0.672001,0.327999,"A good summary is coherent, accurate, covers t..."
1,Resuma a fábula da Cigarra e da Formiga.,"A formiga trabalhou no verão, enquanto a cigar...",A história fala sobre uma cigarra que gostava ...,0.676235,0.323765,"A good summary is coherent, accurate, covers t..."
